In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Llama 3.2 Evaluation

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_llama3_2_evaluation.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_llama3_2_evaluation.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates evaluating text and multimodal Llama 3.2 models in Vertex AI.

### Objective

- Evaluate text and multimodal Llama 3.2 models using [lm-evaluation-harness](https://github.com/EleutherAI/lm-evaluation-harness)
- Clean up resources

| Models |
| :- |
| [meta-llama/Llama-3.2-1B](https://huggingface.co/meta-llama/Llama-3.2-1B)
| [meta-llama/Llama-3.2-1B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct)
| [meta-llama/Llama-3.2-3B](https://huggingface.co/meta-llama/Llama-3.2-3B)
| [meta-llama/Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)
| [meta-llama/Llama-3.2-11B-Vision](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision)
| [meta-llama/Llama-3.2-11B-Vision-Instruct](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct)
| [meta-llama/Llama-3.2-90B-Vision](https://huggingface.co/meta-llama/Llama-3.2-90B-Vision)
| [meta-llama/Llama-3.2-90B-Vision-Instruct](https://huggingface.co/meta-llama/Llama-3.2-90B-Vision-Instruct)

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the notebook

In [ ]:
# @title Setup Google Cloud project
# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

BUCKET_URI = "gs://"  # @param {type:"string"}

# @markdown 3. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

# Import the necessary packages

! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

import datetime
import importlib
import os
import re
import uuid

from google.cloud import aiplatform
from google.cloud.aiplatform.compat.types import \
    custom_job as gca_custom_job_compat

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)

models, endpoints = {}, {}

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    if not os.environ.get("GOOGLE_CLOUD_REGION"):
        raise ValueError(
            "REGION must be set. See"
            " https://cloud.google.com/vertex-ai/docs/general/locations for"
            " available cloud locations."
        )
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "llama3_2")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

# @markdown Provide a Hugging Face User Access Token (read) for an account that has access to the Llama 3.2 models. You can follow the [Hugging Face documentation](https://huggingface.co/docs/hub/en/security-tokens) to create a **read** access token and put it in the `HF_TOKEN` field below.
HF_TOKEN = ""  # @param {type:"string", isTemplate:true}
assert (
    HF_TOKEN
), "Provide a read HF_TOKEN to load restricted access models from Hugging Face."

In [ ]:
# @title Evaluate Llama 3.2 models

# @markdown This section demonstrates how to evaluate Llama 3.2 text and multimodal models using EleutherAI's [Language Model Evaluation Harness (lm-evaluation-harness)](https://github.com/EleutherAI/lm-evaluation-harness) with Vertex CustomJob.

# @markdown Set the base model id.
base_model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"  # @param["meta-llama/Llama-3.2-1B", "meta-llama/Llama-3.2-1B-Instruct", "meta-llama/Llama-3.2-3B", "meta-llama/Llama-3.2-3B-Instruct", "meta-llama/Llama-3.2-11B-Vision", "meta-llama/Llama-3.2-11B-Vision-Instruct", "meta-llama/Llama-3.2-90B-Vision", "meta-llama/Llama-3.2-90B-Vision-Instruct"] {isTemplate:true}
job_name = common_util.get_job_name_with_datetime(prefix="llama3.2-eval")
eval_output_dir = os.path.join(MODEL_BUCKET, job_name)
eval_output_dir_gcsfuse = eval_output_dir.replace("gs://", "/gcs/")

# @markdown This example uses the [HellaSwag](https://arxiv.org/abs/1905.07830) dataset for text models and [MMMU](https://mmmu-benchmark.github.io/) for multimodal models. All supported tasks are listed in [this directory](https://github.com/EleutherAI/lm-evaluation-harness/tree/main/lm_eval/tasks).
eval_dataset = "mmmu_val"  # @param ["mmmu_val", "hellaswag"] {allow-input: true, isTemplate: true}
if "Vision" not in base_model_id and eval_dataset == "mmmu_val":
    raise ValueError(
        f"MMMU is a multimodal evaluation benchmark and can't be used with {base_model_id}."
    )

# @markdown Find Vertex AI supported accelerators and regions in: https://cloud.google.com/vertex-ai/docs/training/configure-compute
if "3.2-1B" in base_model_id or "3.2-3B" in base_model_id:
    accelerator_type = "NVIDIA_L4"
    machine_type = "g2-standard-12"
    accelerator_count = 1
    eval_type = "vllm"
    dws_kwargs = {}
    is_dynamic_workload_scheduler = False
    model_args = f"pretrained={base_model_id},tensor_parallel_size={accelerator_count},swap_space=16,gpu_memory_utilization=0.95,max_num_seqs={64 if '3.2-3B' in base_model_id else 256}"
elif "3.2-11B" in base_model_id:
    accelerator_type = "NVIDIA_H100_80GB"
    machine_type = "a3-highgpu-1g"
    accelerator_count = 1
    eval_type = "vllm-vlm"
    dws_kwargs = {
        "max_wait_duration": 1800,  # 30 minutes
        "scheduling_strategy": gca_custom_job_compat.Scheduling.Strategy.FLEX_START,
    }
    is_dynamic_workload_scheduler = True
    model_args = f"pretrained={base_model_id},tensor_parallel_size={accelerator_count},swap_space=16,gpu_memory_utilization=0.95,enforce_eager=True,max_num_seqs=12,max_model_len=8192,max_images=1"
elif "3.2-90B" in base_model_id:
    accelerator_type = "NVIDIA_H100_80GB"
    machine_type = "a3-highgpu-8g"
    accelerator_count = 8
    eval_type = "vllm-vlm"
    dws_kwargs = {
        "max_wait_duration": 1800,  # 30 minutes
        "scheduling_strategy": gca_custom_job_compat.Scheduling.Strategy.FLEX_START,
    }
    is_dynamic_workload_scheduler = True
    model_args = f"pretrained={base_model_id},tensor_parallel_size={accelerator_count},swap_space=16,gpu_memory_utilization=0.95,enforce_eager=True,max_num_seqs=22,max_images=1"
else:
    raise ValueError(f"Recommended GPU setting not found for: {base_model_id}.")

replica_count = 1

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    is_for_training=True,
    is_dynamic_workload_scheduler=is_dynamic_workload_scheduler,
)

# Prepare evaluation command that runs the evaluation harness.
eval_command = [
    "lm_eval",
    "--model",
    eval_type,
    "--tasks",
    eval_dataset,
    "--output_path",
    f"{eval_output_dir_gcsfuse}",
    "--model_args",
    model_args,
]

if "Instruct" in base_model_id:
    eval_command.append("--apply_chat_template")

# The evaluation docker image.
EVAL_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-lm-evaluation-harness:20241016_0934_RC00"

# Pass evaluation arguments and launch job.
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        },
        "replica_count": replica_count,
        "disk_spec": {
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": EVAL_DOCKER_URI,
            "env": [
                {
                    "name": "HF_TOKEN",
                    "value": HF_TOKEN,
                }
            ],
            "command": eval_command,
            "args": [],
        },
    }
]


# Add labels for the finetuning job.
labels = {
    "mg-source": "notebook",
    "mg-notebook-name": "model_garden_llama3_2_evaluation.ipynb".split(".")[0],
}

labels["mg-tune"] = "publishers-meta-models-llama3-2"
versioned_model_id = base_model_id.split("/")[1].lower().replace(".", "-")
labels["versioned-mg-tune"] = f"{labels['mg-tune']}-{versioned_model_id}"

eval_job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    base_output_dir=eval_output_dir,
    labels=labels,
)

eval_job.run(**dws_kwargs)

print("Evaluation results were saved in:", eval_output_dir)

In [ ]:
# @title Fetch and print evaluation results
import json

from google.cloud import storage

# Fetch evaluation results.
storage_client = storage.Client()
BUCKET_NAME = BUCKET_URI.split("gs://")[1]
bucket = storage_client.get_bucket(BUCKET_NAME)

blobs = [b.name for b in bucket.list_blobs()]

result_file_path = None
for file_path in filter(re.compile(".*/*.json").match, blobs):
    result_file_path = file_path
    print(f"Found result file: {file_path}")

if result_file_path is None:
    raise ValueError("No result file found.")

blob = bucket.blob(result_file_path)
raw_result = blob.download_as_string()

# Print evaluation results.
result = json.loads(raw_result)
result_formatted = json.dumps(result, indent=2)
print(f"Evaluation result:\n{result_formatted}")

## Clean up resources

In [ ]:
# Delete evaluation job.

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI
    # Uncomment below to delete all artifacts
    # !gsutil -m rm -r $STAGING_BUCKET $MODEL_BUCKET $EXPERIMENT_BUCKET

eval_job.delete()